<a href="https://colab.research.google.com/github/andremonroy/stanWeinstein/blob/main/Obtain_sector%2C_weighted_RS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RS con formula simple

In [ ]:
# --- Install dependencies (only needed in Colab) ---
!pip install yfinance pandas numpy scikit-learn --quiet

import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# -----------------------
# 1. INPUT: Your tickers
# -----------------------
tickers = ['FOFO', 'EMPG', 'BKV', 'IREN', 'SEI', 'STOK', 'KRMN', 'EGO', 'RYTM', 'TSM', 'APP', 'KGC', 'TTWO', 'TVTX', 'FNV', 'IBKR', 'LIF', 'MRCY', 'AEM', 'EME', 'RIOT', 'WPM', 'ARQT', 'HG', 'TFPM', 'HIMS', 'NVDA', 'FIX', 'AU', 'BTSG', 'RMBS', 'NVMI', 'INOD', 'MEDP', 'CCJ', 'HSAI', 'TBBK', 'APH', 'GFI', 'MIRM', 'HOOD', 'ONC', 'PAHC', 'PLTR', 'FUTU', 'GOOGL', 'MU', 'STNE', 'CLS', 'ANIP', 'AEIS', 'ANET', 'AMSC', 'SOFI', 'HALO', 'BZ', 'AVDL', 'ATAT', 'ALNT', 'SITM', 'DAVE', 'SYM', 'EXTR', 'TIGR', 'RDDT', 'WGS', 'PTRN', 'ALAB']

# Sector ETFs (SPDR) - Added more relevant ETFs for better mapping
sector_etfs = {
    "XLC": "Communication Services",
    "XLY": "Consumer Discretionary",
    "XLP": "Consumer Staples",
    "XLE": "Energy",
    "XLF": "Financials",
    "XLV": "Health Care",
    "XLI": "Industrials",
    "XLB": "Materials",
    "XLRE": "Real Estate",
    "XLK": "Technology",
    "XLU": "Utilities",
    "IYH": "Healthcare", # iShares U.S. Healthcare
    "IYK": "Consumer Staples", # iShares U.S. Consumer Staples
    "IYC": "Consumer Discretionary", # iShares U.S. Consumer Discretionary
    "IYF": "Financials", # iShares U.S. Financials
    "IYE": "Energy", # iShares U.S. Energy
    "IYJ": "Industrials", # iShares U.S. Industrials
    "IYW": "Technology", # iShares U.S. Technology
    "IYZ": "Telecommunications", # iShares U.S. Telecommunications
    "IYM": "Materials", # iShares U.S. Basic Materials
    "IYR": "Real Estate", # iShares U.S. Real Estate
    "IDU": "Utilities", # iShares U.S. Utilities
    "IBB": "Biotechnology", # iShares Nasdaq Biotechnology
    "KCE": "Capital Markets", # SPDR S&P Capital Markets ETF
    "KIE": "Insurance", # SPDR S&P Insurance ETF
    "KRE": "Regional Banks", # SPDR S&P Regional Banking ETF
    "XSD": "Semiconductor", # SPDR S&P Semiconductor ETF
    "XBI": "Biotechnology", # SPDR S&P Biotech ETF
    "XHE": "Healthcare Equip", # SPDR S&P Healthcare Equipment ETF
    "XHS": "Healthcare Services", # SPDR S&P Healthcare Services ETF
    "XPH": "Pharmaceuticals", # SPDR S&P Pharmaceuticals ETF - Corrected Ticker
    "XRT": "Retail", # SPDR S&P Retail ETF
    "FDN": "Internet", # First Trust Dow Jones Internet Index Fund
    "SKYY": "Cloud Computing", # First Trust Cloud Computing ETF
    "CIBR": "Cybersecurity", # First Trust NASDAQ Cybersecurity ETF

}


# ----------------------------
# 2. Map ticker -> sector ETF
# ----------------------------
ticker_info = {}
for t in tickers:
    sector = "Unknown"
    mapped_etf = None
    try:
        info = yf.Ticker(t).info
        sector = info.get("sector", "Unknown")

        # Attempt exact match first
        for etf, sname in sector_etfs.items():
            if sector and sname.lower() == sector.lower():
                mapped_etf = etf
                break

        # If no exact match, attempt partial match
        if mapped_etf is None:
             for etf, sname in sector_etfs.items():
                if sector and sname.lower() in sector.lower():
                    mapped_etf = etf
                    break

        # Fallback for specific sectors or tickers if needed (can be expanded)
        if mapped_etf is None:
             if "Healthcare" in sector:
                 mapped_etf = "XLV" # Default to general Healthcare ETF
             elif "Financial" in sector:
                 mapped_etf = "XLF" # Default to general Financials ETF
             # Add more fallbacks here as needed


    except Exception:
        sector = "Unknown"
        mapped_etf = None # Ensure mapped_etf is None on error

    ticker_info[t] = {"sector": sector, "etf": mapped_etf}

# ----------------------------
# 3. Download price data
# ----------------------------
# Include all potential ETFs for download
all_symbols = list(set(tickers + list(sector_etfs.keys()) + ["SPY"]))
data = yf.download(all_symbols, start="2024-01-01", interval="1wk", auto_adjust=True)

# Handle both flat and multi-index DataFrames
if isinstance(data.columns, pd.MultiIndex):
    prices = data["Close"]
else:
    prices = data[["Close"]] if "Close" in data.columns else data


# ----------------------------
# 4. Calculate RS vs SPY for tickers and sector ETFs
# ----------------------------
rs_ticker_values = {}
rs_etf_values = {}

for t in tickers:
    # Calculate RS for individual ticker
    if t in prices.columns:
        try:
            rs_ticker = prices[t] / prices["SPY"]
            latest_rs_ticker = rs_ticker.dropna().iloc[-1]
            rs_ticker_values[t] = latest_rs_ticker
        except:
            rs_ticker_values[t] = np.nan
    else:
        rs_ticker_values[t] = np.nan

    # Calculate RS for mapped sector ETF
    etf = ticker_info[t]["etf"]
    if etf and etf in prices.columns:
        try:
            rs_etf = prices[etf] / prices["SPY"]
            latest_rs_etf = rs_etf.dropna().iloc[-1]
            rs_etf_values[t] = latest_rs_etf
        except:
            rs_etf_values[t] = np.nan
    else:
        rs_etf_values[t] = np.nan


# Normalize RS to 0-100 scale for tickers
scaler_ticker = MinMaxScaler(feature_range=(0,100))
rs_ticker_df = pd.DataFrame.from_dict(rs_ticker_values, orient="index", columns=["RS_ticker_raw"])
rs_ticker_df_cleaned = rs_ticker_df.dropna(subset=["RS_ticker_raw"])

if not rs_ticker_df_cleaned.empty:
    rs_ticker_df["RS_ticker"] = scaler_ticker.fit_transform(rs_ticker_df[["RS_ticker_raw"]])
else:
    rs_ticker_df["RS_ticker"] = np.nan

# Normalize RS to 0-100 scale for sector ETFs
scaler_etf = MinMaxScaler(feature_range=(0,100))
rs_etf_df = pd.DataFrame.from_dict(rs_etf_values, orient="index", columns=["RS_etf_raw"])
rs_etf_df_cleaned = rs_etf_df.dropna(subset=["RS_etf_raw"])

if not rs_etf_df_cleaned.empty:
    rs_etf_df["RS_sector_etf"] = scaler_etf.fit_transform(rs_etf_df[["RS_etf_raw"]])
else:
    rs_etf_df["RS_sector_etf"] = np.nan

# Display raw RS for sector ETFs
print("Raw RS values for Sector ETFs:")
display(rs_etf_df)


# ----------------------------
# 5. Combine results
# ----------------------------
results = []
for t in tickers:
    sec = ticker_info[t]["sector"]
    etf = ticker_info[t]["etf"]
    rs_ticker = rs_ticker_df.loc[t, "RS_ticker"] if t in rs_ticker_df.index and not pd.isna(rs_ticker_df.loc[t, "RS_ticker"]) else np.nan
    rs_sector_etf = rs_etf_df.loc[t, "RS_sector_etf"] if t in rs_etf_df.index and not pd.isna(rs_etf_df.loc[t, "RS_sector_etf"]) else np.nan
    results.append([t, sec, etf, rs_ticker, rs_sector_etf])

df = pd.DataFrame(results, columns=["Ticker","Sector","Sector ETF","RS Ticker Score","RS Sector ETF Score"])

# Sort by RS Sector ETF Score (descending) then by RS Ticker Score (descending)
df = df.sort_values(["RS Sector ETF Score", "RS Ticker Score"], ascending=[False, False])


# ----------------------------
# 6. Display ranked table
# ----------------------------
print("\nRelative Strength Ranking (Sorted by Sector ETF RS then Ticker RS):")
print(df.to_string(index=False))

[*********************100%***********************]  104 of 104 completed

Raw RS values for Sector ETFs:


,RS_etf_raw,RS_sector_etf
FOFO,NaN,NaN
EMPG,0.086337,1.401414
BKV,0.136060,15.990276
IREN,0.081560,0.000000
SEI,0.136060,15.990276
...,...,...
TIGR,0.081560,0.000000
RDDT,0.178625,28.479225
WGS,0.086337,1.401414
PTRN,NaN,NaN



Relative Strength Ranking (Sorted by Sector ETF RS then Ticker RS):
Ticker                 Sector Sector ETF  RS Ticker Score  RS Sector ETF Score
  NVMI             Technology        XLK        37.958632           100.000000
  SITM             Technology        XLK        35.684124           100.000000
   TSM             Technology        XLK        33.171924           100.000000
   CLS             Technology        XLK        29.722862           100.000000
  DAVE             Technology        XLK        25.287632           100.000000
  ALAB             Technology        XLK        23.761601           100.000000
  NVDA             Technology        XLK        21.618582           100.000000
  PLTR             Technology        XLK        21.240550           100.000000
    MU             Technology        XLK        19.349128           100.000000
  ANET             Technology        XLK        16.753483           100.000000
   APH             Technology        XLK        13.926468     

# Task
Calculate a Relative Strength (RS Rating) indicator similar to MarketSmith for a set of stocks, following these rules:
- Use the percentage returns of each stock over the last 3, 6, and 12 months.
- Assign a higher weight to recent periods (e.g., 40% for 3 months, 30% for 6 months, 30% for 12 months).
- Combine these results into a single relative performance score.
- Order all stocks and convert them into a percentile rank from 1 to 99, where 99 is the top 1% of performance against the universe and 1 is the worst.
- Calculate the RS using the same strategy for both individual tickers and sector ETFs.
- The final order should be based on the sector ETF's RS first, and then the individual ticker's RS.

## Descarga de datos

### Subtask:
Descargar los precios históricos de cierre para los tickers especificados, los ETFs sectoriales y el SPY, asegurando datos suficientes para calcular retornos de 3, 6 y 12 meses.


**Reasoning**:
Download historical adjusted close prices for the specified tickers, sector ETFs, and SPY using a start date that allows for 12-month return calculations and a weekly interval. Extract the 'Close' prices into a DataFrame named `prices`.



In [ ]:
# 3. Download price data
# ----------------------------
all_symbols = list(set(tickers + list(sector_etfs.keys()) + ["SPY"]))
data = yf.download(all_symbols, start="2023-09-01", interval="1wk", auto_adjust=True)

# Handle both flat and multi-index DataFrames
if isinstance(data.columns, pd.MultiIndex):
    prices = data["Close"]
else:
    prices = data[["Close"]] if "Close" in data.columns else data

display(prices.head())

[*********************100%***********************]  104 of 104 completed


Ticker,AEIS,AEM,ALAB,ALNT,AMSC,ANET,ANIP,APH,APP,ARQT,...,XLI,XLK,XLP,XLRE,XLU,XLV,XLY,XPH,XRT,XSD
Date,,,,,,,,,,,,,,,,,,,,,
2023-08-28,117.323433,45.783932,NaN,35.523525,9.36,49.340000,64.580002,43.459274,43.189999,8.96,...,105.107597,173.426346,67.872322,34.289246,58.439629,128.653534,166.563507,42.767025,62.741333,212.948822
2023-09-04,103.381645,45.430790,NaN,32.083199,8.06,49.117500,61.939999,42.144257,42.119999,7.42,...,102.045189,170.070023,67.458351,33.937084,58.943420,127.199097,165.660370,41.681366,60.187386,201.867065
2023-09-11,103.848366,47.070793,NaN,31.171072,7.72,46.180000,62.389999,42.060848,42.820000,6.81,...,101.436569,166.241272,67.787643,34.094631,60.585407,127.324326,168.644714,41.138538,60.430157,198.348724
2023-09-18,98.923065,47.042015,NaN,30.239113,7.62,45.084999,58.520000,40.888130,37.759998,5.38,...,98.335495,161.821960,65.990639,32.037270,59.046036,125.359375,157.983536,39.267731,57.866508,191.431320
2023-09-25,102.398575,43.589386,NaN,30.655519,7.55,45.982498,58.060001,41.313183,39.959999,5.31,...,98.338394,161.694153,65.178062,31.825644,55.423904,124.509933,158.313705,38.640289,59.262627,194.948242


## Mapeo de ticker a etf sectorial

### Subtask:
Utilizar la información disponible para mapear cada ticker a su ETF sectorial correspondiente.


## Cálculo de retornos

### Subtask:
Calcular los retornos porcentuales para cada ticker, cada ETF sectorial (para los que se pudo mapear un ticker) y para el SPY en los períodos de 3, 6 y 12 meses.


**Reasoning**:
Calculate the percentage change for each asset in the prices DataFrame and then calculate the cumulative returns for the last 3, 6, and 12 months. Store these returns in dictionaries.



In [ ]:
# Calculate percentage change for each asset
returns = prices.pct_change()

# Define the time periods in weeks (approximate: 1 month = 4 weeks)
periods = {
    "3m": 13,
    "6m": 26,
    "12m": 52,
}

# Calculate cumulative returns for each period for tickers, mapped ETFs, and SPY
cumulative_returns = {}

for symbol in all_symbols:
    if symbol in returns.columns:
        cumulative_returns[symbol] = {}
        for period_name, weeks in periods.items():
            # Ensure enough data points for the period
            if len(returns[symbol]) >= weeks:
                cumulative_returns[symbol][period_name] = (1 + returns[symbol].iloc[-weeks:]).prod() - 1
            else:
                cumulative_returns[symbol][period_name] = np.nan

# Display the calculated cumulative returns
print("Cumulative Returns:")
display(cumulative_returns)

Cumulative Returns:


{'ALNT': {'3m': np.float64(0.11765037841127857),
  '6m': np.float64(1.1297739428723501),
  '12m': np.float64(1.4270420597730187)},
 'SYM': {'3m': np.float64(0.12147776483740902),
  '6m': np.float64(2.0702856881277913),
  '12m': np.float64(1.2537751341584538)},
 'AVDL': {'3m': np.float64(0.7144456770895187),
  '6m': np.float64(1.2514705867915836),
  '12m': np.float64(0.1695951291940907)},
 'SITM': {'3m': np.float64(0.3770908247818683),
  '6m': np.float64(1.4582739063260366),
  '12m': np.float64(0.6338703779516754)},
 'APH': {'3m': np.float64(0.218356030600118),
  '6m': np.float64(1.0543608930868023),
  '12m': np.float64(0.9547074447058979)},
 'MU': {'3m': np.float64(0.3415182304276294),
  '6m': np.float64(1.5381312793279474),
  '12m': np.float64(0.6104302584303263)},
 'IYF': {'3m': np.float64(0.03726775604642851),
  '6m': np.float64(0.28174135351621676),
  '12m': np.float64(0.23134914648379756)},
 'ANET': {'3m': np.float64(0.39845883546352856),
  '6m': np.float64(1.2272796178309968),
  

## Cálculo de fuerza relativa ponderada para tickers

### Subtask:
Calcular la fuerza relativa para cada ticker en cada período (retorno del ticker dividido por el retorno del SPY) y luego combinarlos utilizando las ponderaciones especificadas (40% para 3 meses, 30% para 6 meses, 30% para 12 meses) para obtener un puntaje de RS ponderado para cada ticker.


**Reasoning**:
Calculate the weighted RS scores for each ticker using the cumulative returns and specified weights, and then display the results in a DataFrame.



In [ ]:
# 1. Define the weights for each period
weights = {"3m": 0.4, "6m": 0.3, "12m": 0.3}

# 2. Initialize an empty dictionary to store the weighted RS scores for each ticker
weighted_rs_ticker_scores = {}

# 3. Iterate through the list of tickers
for t in tickers:
    # 4. Calculate the relative strength for each period
    rs_periods = {}
    for period_name, weight in weights.items():
        ticker_return = cumulative_returns.get(t, {}).get(period_name)
        spy_return = cumulative_returns.get("SPY", {}).get(period_name)

        if spy_return is not None and not np.isnan(spy_return) and spy_return != 0 and ticker_return is not None and not np.isnan(ticker_return):
            rs_periods[period_name] = ticker_return / spy_return
        else:
            rs_periods[period_name] = np.nan

    # 5. Calculate the weighted RS score
    weighted_rs = np.nan
    if not any(np.isnan(list(rs_periods.values()))):
        weighted_rs = sum(rs_periods[period_name] * weights[period_name] for period_name in weights)

    # 6. Store the calculated weighted RS score
    weighted_rs_ticker_scores[t] = weighted_rs

# 7. Convert the dictionary to a pandas DataFrame
rs_ticker_weighted_df = pd.DataFrame.from_dict(weighted_rs_ticker_scores, orient="index", columns=["RS_ticker_weighted"])

# 8. Display the resulting DataFrame
print("\nWeighted RS scores for each ticker:")
display(rs_ticker_weighted_df)


Weighted RS scores for each ticker:


,RS_ticker_weighted
FOFO,20.157114
EMPG,30.661520
BKV,1.181648
IREN,24.534297
SEI,6.846711
...,...
TIGR,1.234598
RDDT,9.113225
WGS,4.152343
PTRN,-0.985122


## Cálculo de fuerza relativa ponderada para etfs sectoriales

### Subtask:
Calcular la fuerza relativa para cada ETF sectorial (para los ETFs mapeados) en cada período (retorno del ETF sectorial dividido por el retorno del SPY) y luego combinarlos utilizando las ponderaciones especificadas para obtener un puntaje de RS ponderado para cada ETF sectorial.


**Reasoning**:
Calculate the weighted Relative Strength (RS) score for each mapped sector ETF using the cumulative returns and specified weights, then store and display the results in a DataFrame.



In [ ]:
# 1. Initialize an empty dictionary to store the weighted RS scores for each mapped sector ETF.
weighted_rs_etf_scores = {}

# Iterate through the ticker_info to get unique mapped ETFs
processed_etfs = set()

for t, info in ticker_info.items():
    etf = info.get("etf")

    # 2. For each mapped sector ETF, calculate the relative strength for each period
    if etf and etf in cumulative_returns and etf not in processed_etfs:
        rs_periods_etf = {}
        for period_name, weight in weights.items():
            etf_return = cumulative_returns.get(etf, {}).get(period_name)
            spy_return = cumulative_returns.get("SPY", {}).get(period_name)

            if spy_return is not None and not np.isnan(spy_return) and spy_return != 0 and etf_return is not None and not np.isnan(etf_return):
                rs_periods_etf[period_name] = etf_return / spy_return
            else:
                rs_periods_etf[period_name] = np.nan

        # 4. Calculate the weighted RS score for the sector ETF
        weighted_rs_etf = np.nan
        if not any(np.isnan(list(rs_periods_etf.values()))):
            weighted_rs_etf = sum(rs_periods_etf[period_name] * weights[period_name] for period_name in weights)

        # 5. Store the calculated weighted RS score.
        weighted_rs_etf_scores[etf] = weighted_rs_etf
        processed_etfs.add(etf) # Mark as processed to avoid recalculation

# 6. Convert the dictionary resulting of weighted RS scores of sector ETFs to a pandas DataFrame.
rs_etf_weighted_df = pd.DataFrame.from_dict(weighted_rs_etf_scores, orient="index", columns=["RS_sector_etf_weighted"])

# 7. Display the resulting DataFrame.
print("\nWeighted RS scores for each mapped Sector ETF:")
display(rs_etf_weighted_df)


Weighted RS scores for each mapped Sector ETF:


,RS_sector_etf_weighted
IYH,-0.068783
XLE,0.446198
XLF,0.702144
XLI,0.719968
XLB,-0.051203
XLK,1.520055
XLC,1.528526


## Combinación de resultados

### Subtask:
Consolidar los puntajes de RS ponderados de los tickers y los ETFs sectoriales en una única tabla.


**Reasoning**:
Merge the dataframes containing ticker information, weighted ticker RS, and weighted sector ETF RS, handle missing values, select and reorder columns, and display the result.



In [ ]:
# Merge df with rs_ticker_weighted_df based on Ticker
merged_df = pd.merge(df, rs_ticker_weighted_df, left_on='Ticker', right_index=True, how='left')

# Merge the result with rs_etf_weighted_df based on Sector ETF
final_df = pd.merge(merged_df, rs_etf_weighted_df, left_on='Sector ETF', right_index=True, how='left')

# Select and reorder columns
final_df = final_df[['Ticker', 'Sector', 'Sector ETF', 'RS_ticker_weighted', 'RS_sector_etf_weighted']]

# Handle potential missing values (fill with NaN, which is the default for left merge)
# No explicit action needed here as left merge with default fill handles NaNs.

# Display the combined DataFrame
print("\nConsolidated Relative Strength Scores:")
display(final_df)


Consolidated Relative Strength Scores:


,Ticker,Sector,Sector ETF,RS_ticker_weighted,RS_sector_etf_weighted
31,NVMI,Technology,XLK,2.362501,1.520055
59,SITM,Technology,XLK,4.813473,1.520055
9,TSM,Technology,XLK,2.742871,1.520055
48,CLS,Technology,XLK,12.140121,1.520055
60,DAVE,Technology,XLK,7.062488,1.520055
...,...,...,...,...,...
63,TIGR,Financial Services,XLF,1.234598,0.702144
0,FOFO,Unknown,None,20.157114,NaN
57,ATAT,Consumer Cyclical,None,1.806934,NaN
35,HSAI,Consumer Cyclical,None,10.003664,NaN


## Ranking percentil

### Subtask:
Order the combined table first by the weighted RS score of the sector ETF (from highest to lowest) and then by the weighted RS score of the individual ticker (from highest to lowest). Convert these scores to a percentile rank from 1 to 99.


**Reasoning**:
Sort the dataframe by the weighted RS scores and then calculate the percentile ranks for both the ticker and sector ETF weighted RS scores, scaling them to 1-99.



In [ ]:
# Sort the DataFrame
final_df = final_df.sort_values(
    ["RS_sector_etf_weighted", "RS_ticker_weighted"], ascending=[False, False]
)

# Calculate percentile rank for 'RS_ticker_weighted' (scaled to 1-99)
# Rank based on non-NaN values, then map back to the original DataFrame
final_df["RS Ticker Percentile Rank"] = (
    final_df["RS_ticker_weighted"].rank(method="min", pct=True) * 99
).apply(lambda x: 1 if pd.isna(x) else max(1, round(x)))


# Calculate percentile rank for 'RS_sector_etf_weighted' (scaled to 1-99)
# Rank based on non-NaN values, then map back to the original DataFrame
final_df["RS Sector ETF Percentile Rank"] = (
    final_df["RS_sector_etf_weighted"].rank(method="min", pct=True) * 99
).apply(lambda x: 1 if pd.isna(x) else max(1, round(x)))

# Display the updated DataFrame
print("\nRelative Strength Ranking (Sorted and with Percentile Ranks):")
display(final_df)


Relative Strength Ranking (Sorted and with Percentile Ranks):


,Ticker,Sector,Sector ETF,RS_ticker_weighted,RS_sector_etf_weighted,RS Ticker Percentile Rank,RS Sector ETF Percentile Rank
10,APP,Communication Services,XLC,15.992090,1.528526,95,93
64,RDDT,Communication Services,XLC,9.113225,1.528526,83,93
45,GOOGL,Communication Services,XLC,3.693315,1.528526,32,93
55,BZ,Communication Services,XLC,2.775511,1.528526,17,93
12,TTWO,Communication Services,XLC,2.070859,1.528526,10,93
...,...,...,...,...,...,...,...
54,HALO,Healthcare,IYH,2.850180,-0.068783,19,2
0,FOFO,Unknown,None,20.157114,NaN,96,1
35,HSAI,Consumer Cyclical,None,10.003664,NaN,86,1
57,ATAT,Consumer Cyclical,None,1.806934,NaN,7,1


# RS con formula ponderada, parecida a Marketsmith
## Presentación de resultados

### Subtask:
Present the final table with the tickers, their sector ETFs, their weighted RS scores, and their percentile ranks.


**Reasoning**:
Present the final ranked table with the required columns and formatting.



In [ ]:
# Print a descriptive header
print("\nFinal Relative Strength Ranking (Sorted by Sector ETF RS then Ticker RS):")

# Print the final_df DataFrame to the console with all columns and no index
print(final_df.to_string(index=False))


Final Relative Strength Ranking (Sorted by Sector ETF RS then Ticker RS):
Ticker                 Sector Sector ETF  RS_ticker_weighted  RS_sector_etf_weighted  RS Ticker Percentile Rank  RS Sector ETF Percentile Rank
   APP Communication Services        XLC           15.992090                1.528526                         95                             93
  RDDT Communication Services        XLC            9.113225                1.528526                         83                             93
 GOOGL Communication Services        XLC            3.693315                1.528526                         32                             93
    BZ Communication Services        XLC            2.775511                1.528526                         17                             93
  TTWO Communication Services        XLC            2.070859                1.528526                         10                             93
  ALAB             Technology        XLK           14.849371       

## Summary:

### Data Analysis Key Findings

*   The analysis successfully calculated a weighted Relative Strength (RS) score for each individual ticker and its corresponding sector ETF. The weighting applied 40% to the 3-month return, 30% to the 6-month return, and 30% to the 12-month return, relative to the SPY.
*   The tickers and sector ETFs were ranked based on these weighted RS scores. The primary sorting was by the sector ETF's weighted RS (highest to lowest), followed by the individual ticker's weighted RS (highest to lowest).
*   Percentile ranks from 1 to 99 were assigned to both the individual ticker's and the sector ETF's weighted RS scores, providing a standardized measure of performance relative to the universe of analyzed assets.

### Insights or Next Steps

*   The calculated RS ratings and percentile ranks provide a quantitative basis for comparing the relative performance of individual stocks and their sectors.
*   This analysis can be extended by incorporating other factors like fundamental data or technical indicators to refine investment decisions.
